In [1]:
# Some basics are installed/imported for you
try:
  import pyvo as vo
except ImportError:
  %pip install pyvo
  import pyvo as vo

import numpy as np
from astropy.table import Table
import matplotlib as plt

from io import BytesIO
from astropy.io import votable

import utils

In [2]:
# Initiate the SIMBAD service
r = utils.SIMBAD(vo)

In [3]:
table = utils.table_info(r, 'basic')

In [4]:
basic_cols = table.get_column_names()
print('Columns in basic table:', basic_cols)

Columns in basic table: ['coo_bibcode', 'coo_err_angle', 'coo_err_maj', 'coo_err_maj_prec', 'coo_err_min', 'coo_err_min_prec', 'coo_qual', 'coo_wavelength', 'dec_prec', 'galdim_angle', 'galdim_bibcode', 'galdim_majaxis', 'galdim_majaxis_prec', 'galdim_minaxis', 'galdim_minaxis_prec', 'galdim_qual', 'hpx', 'morph_bibcode', 'morph_qual', 'morph_type', 'nbref', 'oid', 'otype', 'plx_bibcode', 'plx_err', 'plx_err_prec', 'plx_prec', 'plx_qual', 'plx_value', 'pm_bibcode', 'pm_err_angle', 'pm_err_maj', 'pm_err_maj_prec', 'pm_err_min', 'pm_err_min_prec', 'pm_qual', 'pmdec', 'pmdec_prec', 'pmra', 'pmra_prec', 'ra_prec', 'rvz_bibcode', 'rvz_err', 'rvz_err_prec', 'rvz_nature', 'rvz_qual', 'rvz_radvel', 'rvz_radvel_prec', 'rvz_redshift', 'rvz_redshift_prec', 'rvz_type', 'sp_bibcode', 'sp_qual', 'sp_type', 'update_date', 'vlsr', 'vlsr_bibcode', 'vlsr_max', 'vlsr_min', 'vlsr_wavelength', 'main_id', 'otype_txt', 'ra', 'dec']


Get unique values from a column given a table 

In [6]:
column_name = 'main_id'
unique_cols = table.get_unique_values(column_name)
print(len(unique_cols))

50000


/Users/jarugula/opt/miniconda3/envs/astro-env/lib/python3.10/site-packages/pyvo/dal/query.py:322: DALOverflowWarning: Partial result set. Potential causes MAXREC, async storage space, etc.
  warn("Partial result set. Potential causes MAXREC, async storage space, etc.",


# 1. Find the SIMBAD entry for the globular cluster "omega Centauri"
> ### Columns to retrieve: `RA`, `DEC`, `main_id`, and `oid`
> ### Print out the main identifer (`main_id`). Note that it is not "omega Centauri".

In [2]:
# Fill in the appropriate query here
q = """
SELECT basic.OID,
       RA,
       DEC,
       main_id
FROM basic JOIN ident ON oidref = oid
WHERE id = 'omega Centauri'
"""

result1 = SIMBAD_service.search(q).to_table()
print("The SIMBAD main identifier for the cluster is {}".format(result1['main_id'][0]))

def totable(q):
    query = q
    result = SIMBAD_service.search(query).to_table()
    return result



t = totable(q)
t

The SIMBAD main identifier for the cluster is NGC  5139


oid,ra,dec,main_id
,deg,deg,
int64,float64,float64,object
3375707,201.69699999999997,-47.47947222222223,NGC 5139


In [29]:
q = """
SELECT basic.OID,
       RA,
       DEC,
       main_id
FROM basic JOIN ident ON oidref = oid;
LIMIT 10
"""
result1 = SIMBAD_service.search(q).to_table()


/Users/jarugula/opt/miniconda3/envs/astro-env/lib/python3.10/site-packages/pyvo/dal/query.py:322: DALOverflowWarning: Partial result set. Potential causes MAXREC, async storage space, etc.
  warn("Partial result set. Potential causes MAXREC, async storage space, etc.",


In [31]:
result1

oid,ra,dec,main_id
,deg,deg,
int64,float64,float64,object
1,314.92644583333333,37.87465833333333,BD+37 4133
1,314.92644583333333,37.87465833333333,BD+37 4133
1,314.92644583333333,37.87465833333333,BD+37 4133
1,314.92644583333333,37.87465833333333,BD+37 4133
1,314.92644583333333,37.87465833333333,BD+37 4133
1,314.92644583333333,37.87465833333333,BD+37 4133
1,314.92644583333333,37.87465833333333,BD+37 4133
1,314.92644583333333,37.87465833333333,BD+37 4133


# 2a. Do a bibliography search and find the most recent paper that studied the Cluster
> ### Specifically, find the latest paper that has the phrases "tidal radius" and "%omega%Cen" in its `abstract`.
> ### The query should extract all the information necessary to construct a URL to the paper's entry on the ADS (see example in Demo Session 2).

In [11]:
# Fill in the appropriate query here
query = """
SELECT TOP 1 bibcode
FROM ref
WHERE abstract LIKE '%omega%Cen%' AND abstract LIKE '%tidal radius%'
ORDER BY "year" DESC
"""

result2 = SIMBAD_service.search(query).to_table()
print("The url is https://ui.adsabs.harvard.edu/abs/{}".format(result2['bibcode'][0]))


The url is https://ui.adsabs.harvard.edu/abs/2017AJ....153..175C


DALQueryError: Incorrect ADQL query: 1 unresolved identifiers: %omega%Cen% [l.1 c.51 - l.1 c.64]!
  - Unknown column ""%omega%Cen%"" !

In [ ]:
querry = []

# 2b. Finding the tidal radius (approximate angular size) of the cluster
> ### Follow the URL to the PDF of the most recent paper. Consult Table 1 of the paper. Here, you will find the value of the tidal radius of the cluster in arcmin.
> ### Change the value of the variable `radius_arcmin` to the appropriate value below
> ### If you couldn't find the tidal radius, let `` be set to 60 arcmin as below.

In [4]:
#Note that the step of finding the tidal radius is MANUAL! This is the only manual step in the exercises.
radius_arcmin = 57.03 # Change this to the radius you find it in Table 1 of the paper
radius_deg = str(radius_arcmin / 60.)

# 3. Perform a cone search for C-, M-, or K-type stars within `radius_deg` of the cluster centre.
> ### Columns to retrieve: `oid`, `main_id`, and `sp_type` (spectral type of all the stars within the matching radius).
> ### Hint: check that stars are within the radius using `CONTAINS`, AND restrict their spectral types with `(sp_type LIKE 'K%' or sp_type LIKE 'C%' or sp_type LIKE 'M%')`
>
> ### How many sources are in your final table?

In [5]:
# Fill in the appropriate query here
query = """
SELECT oid, main_id, sp_type
FROM basic
WHERE CONTAINS(POINT('ICRS', RA, DEC), CIRCLE('ICRS', 201.697, -47.47947222222223, """ + radius_deg + """)) = 1
AND sp_type LIKE 'K%' OR sp_type LIKE 'C%' OR sp_type LIKE 'M%'
"""

result3 = SIMBAD_service.search(query).to_table()
# write code below to tell me how many sources there are in result3
len(result3)

/home/rickycow/miniconda3/envs/codeastro/lib/python3.10/site-packages/pyvo/dal/query.py:322: DALOverflowWarning: Partial result set. Potential causes MAXREC, async storage space, etc.
  warn("Partial result set. Potential causes MAXREC, async storage space, etc.",


50000

# 4. Find the 2MASS counterparts, if available, for each source
> ### Check SIMBAD's `ident` table to see if the `oid` corresponding to each row in `result3` has a counterpart in the 2MASS All-Sky Point Source Catalogue.
> ### Hint: for each `oid`, find the row in `ident` such that `ident.id` starts WITH '2MASS J'
>
> ### How many sources from `result3` do NOT have 2MASS counterparts?

In [6]:
# Fill in the appropriate query here
# Collect the 2MASS identifiers into a column named `TMASS_ID`
query = """

"""

# results3 is now uploaded to the SIMBAD TAP server as `mytable`, and it should
# be used in the query above.
votable_object = BytesIO()
votable.writeto(votable.from_table(results3), votable_object)
votable_object.seek(0)

result4 = SIMBAD_service.search(query, uploads = {'mytable': votable_object}).to_table()

# write code below to tell me how many sources from `results3` don't have 2MASS counterparts in `results4`.

# Extra line: modify the 2MASS identifier column in `results4` to remove the "2MASS J" prefix
#             This is done to correctly match the format in the 2MASS point-source catalogue table (see next cell)
result4['TMASS_ID'] = [rr['TMASS_ID'].replace('2MASS J', '') for rr in result4]

NameError: name 'results3' is not defined

# 5. Obtain the 2MASS photometry from VizieR
> ### Select the columns `magJ` and `magK` from the 2MASS point-source catalogue on VizieR (table ID: `'II/246/out'`)
> ### Hint: for each `oid`, find the row in `'II/246/out'` such that `TMASS_ID` from `results4` is equal to `"2MASS"` in `'II/246/out'`

In [ ]:
Vizier_table_id = 'II/246/out'
# Fill in the appropriate query here
query = """

"""

# results3 is now uploaded to the SIMBAD TAP server as `mytable`, and it should
# be used in the query above.
votable_object = BytesIO()
votable.writeto(votable.from_table(results4), votable_object)
votable_object.seek(0)

result5 = Vizier_service.search(query, uploads = {'mytable': votable_object}).to_table()

# write code below to plot a K vs J-K colour-magnitude diagram showing all the sources in result5.


